In [9]:
import numpy as np

class GraphCryptography:
    """
    Implementation of the cryptographic algorithm using Graph Theory
    as described in the research paper.
    """
    
    def __init__(self, adjacency_matrix, key1):
        """
        Initialize the cryptography system.
        
        Args:
            adjacency_matrix: 2D list/array representing the graph's adjacency matrix
            key1: Integer for Caesar cipher shift (e.g., 3 for +3 shift)
        """
        self.adj_matrix = np.array(adjacency_matrix)
        self.key1 = key1
        self.key2 = self._generate_key2_from_graph()
        
    def _generate_key2_from_graph(self):
        """
        Generate key2 from the adjacency matrix by counting connections.
        This follows the paper's example where the graph produces key2 = [4, 2, 1, 3]
        """
        # Count the number of 1s in each column (representing vertex connections)
        key2 = []
        for col_idx in range(self.adj_matrix.shape[1]):
            count = np.sum(self.adj_matrix[:, col_idx])
            key2.append(count)
        return key2
    
    def _caesar_encrypt(self, text):
        """Apply Caesar cipher encryption with key1."""
        result = ""
        for char in text:
            if char.isalpha():
                # Shift character by key1
                base = ord('A') if char.isupper() else ord('a')
                shifted = (ord(char) - base + self.key1) % 26
                result += chr(base + shifted)
            else:
                result += char
        return result
    
    def _caesar_decrypt(self, text):
        """Apply Caesar cipher decryption with key1."""
        result = ""
        for char in text:
            if char.isalpha():
                # Shift character back by key1
                base = ord('A') if char.isupper() else ord('a')
                shifted = (ord(char) - base - self.key1) % 26
                result += chr(base + shifted)
            else:
                result += char
        return result
    
    def _create_matrix(self, text, cols):
        """Create matrix from text with specified number of columns."""
        # Remove spaces for matrix creation
        text = text.replace(" ", "")
        rows = (len(text) + cols - 1) // cols  # Ceiling division
        
        # Pad text if necessary
        padded_text = text.ljust(rows * cols, 'X')
        
        # Create matrix
        matrix = []
        for i in range(rows):
            row = list(padded_text[i * cols:(i + 1) * cols])
            matrix.append(row)
        return matrix
    
    def _read_by_column_order(self, matrix, key_order):
        """Read matrix column by column according to key order."""
        result = ""
        # Sort indices by key values
        sorted_indices = sorted(range(len(key_order)), key=lambda k: key_order[k])
        
        for col_idx in sorted_indices:
            for row in matrix:
                if col_idx < len(row):
                    result += row[col_idx]
        return result
    
    def _read_by_row(self, matrix):
        """Read matrix row by row."""
        result = ""
        for row in matrix:
            result += ''.join(row)
        return result
    
    def encrypt(self, plaintext):
        """
        Encrypt plaintext using the algorithm from the paper.
        
        Args:
            plaintext: String to encrypt
            
        Returns:
            Encrypted cipher text
        """
        # Step 1: Remove spaces and convert to uppercase
        plaintext = plaintext.upper().replace(" ", "")
        
        # Step 2: Apply Caesar cipher with key1
        caesar_encrypted = self._caesar_encrypt(plaintext)
        print(f"After Caesar cipher (+{self.key1}): {caesar_encrypted}")
        
        # Step 3: Write in matrix form using key2 dimensions
        cols = len(self.key2)
        matrix1 = self._create_matrix(caesar_encrypted, cols)
        print(f"\nMatrix 1 (with key2 order {self.key2}):")
        self._print_matrix(matrix1, self.key2)
        
        # Step 4: Read row by row and permute columns
        permuted = self._read_by_column_order(matrix1, self.key2)
        print(f"\nAfter column permutation: {permuted}")
        
        # Step 5: Write permuted text in matrix form again
        matrix2 = self._create_matrix(permuted, cols)
        print(f"\nMatrix 2 (with key2 order {self.key2}):")
        self._print_matrix(matrix2, self.key2)
        
        # Step 6: Read column by column according to key2 order to get final cipher text
        ciphertext = self._read_by_column_order(matrix2, self.key2)
        print(f"\nFinal cipher text: {ciphertext}")
        
        return ciphertext
    
    def decrypt(self, ciphertext):
        """
        Decrypt cipher text using the algorithm from the paper.
        
        Args:
            ciphertext: String to decrypt
            
        Returns:
            Decrypted plain text
        """
        # Step 1: Write cipher text in matrix form
        cols = len(self.key2)
        
        # First, arrange the ciphertext by columns according to key2 order
        matrix1 = self._arrange_by_columns(ciphertext, self.key2)
        print(f"Cipher text in matrix form (arranged by columns):")
        self._print_matrix(matrix1, self.key2)
        
        # Step 2: Read row by row
        intermediate = self._read_by_row(matrix1)
        print(f"\nAfter reading row by row: {intermediate}")
        
        # Step 3: Arrange in matrix form column by column using key2
        matrix2 = self._arrange_by_columns(intermediate, self.key2)
        print(f"\nMatrix after column arrangement:")
        self._print_matrix(matrix2, self.key2)
        
        # Step 4: Read message row by row
        caesar_encrypted = self._read_by_row(matrix2)
        print(f"\nBefore Caesar decryption: {caesar_encrypted}")
        
        # Step 5: Apply Caesar decryption with key1
        plaintext = self._caesar_decrypt(caesar_encrypted)
        print(f"\nDecrypted plain text: {plaintext}")
        
        return plaintext
    
    def _arrange_by_columns(self, text, key_order):
        """Arrange text in matrix by filling columns according to key order."""
        cols = len(key_order)
        rows = len(text) // cols
        
        # Create empty matrix
        matrix = [['' for _ in range(cols)] for _ in range(rows)]
        
        # Sort indices by key values
        sorted_indices = sorted(range(len(key_order)), key=lambda k: key_order[k])
        
        # Fill columns according to sorted order
        text_idx = 0
        for col_idx in sorted_indices:
            for row_idx in range(rows):
                if text_idx < len(text):
                    matrix[row_idx][col_idx] = text[text_idx]
                    text_idx += 1
        
        return matrix
    
    def _print_matrix(self, matrix, key_header=None):
        """Print matrix in a formatted way."""
        if key_header:
            print("   " + "  ".join(map(str, key_header)))
        for row in matrix:
            print("   " + "  ".join(row))


# Example usage based on the paper
if __name__ == "__main__":
    # Define the adjacency matrix from the paper (Figure 1)
    adjacency_matrix = [
        [1, 1, 1, 1],  # V1
        [1, 0, 0, 1],  # V2
        [1, 0, 0, 0],  # V3
        [1, 1, 0, 1]   # V4
    ]
    
    # Key1 for Caesar cipher
    key1 = 4
    
    # Create cryptography instance
    crypto = GraphCryptography(adjacency_matrix, key1)
    
    print("="*60)
    print("GRAPH THEORY CRYPTOGRAPHY")
    print("="*60)
    print(f"\nAdjacency Matrix:")
    print(np.array(adjacency_matrix))
    print(f"\nGenerated Key2 from graph: {crypto.key2}")
    print(f"Key1 (Caesar shift): +{key1}")
    
    # Test with the example from the paper
    plaintext = "THIS IS AN EXAM"
    print(f"\n{'='*60}")
    print("ENCRYPTION")
    print("="*60)
    print(f"\nOriginal plaintext: {plaintext}")
    
    ciphertext = crypto.encrypt(plaintext)
    
    print(f"\n{'='*60}")
    print("DECRYPTION")
    print("="*60)
    print(f"\nCipher text to decrypt: {ciphertext}")
    
    decrypted = crypto.decrypt(ciphertext)
    
    print(f"\n{'='*60}")
    print("VERIFICATION")
    print("="*60)
    print(f"Original:  {plaintext.replace(' ', '')}")
    print(f"Decrypted: {decrypted}")
    print(f"Match: {plaintext.replace(' ', '').upper() == decrypted}")

GRAPH THEORY CRYPTOGRAPHY

Adjacency Matrix:
[[1 1 1 1]
 [1 0 0 1]
 [1 0 0 0]
 [1 1 0 1]]

Generated Key2 from graph: [4, 2, 1, 3]
Key1 (Caesar shift): +4

ENCRYPTION

Original plaintext: THIS IS AN EXAM
After Caesar cipher (+4): XLMWMWERIBEQ

Matrix 1 (with key2 order [4, 2, 1, 3]):
   4  2  1  3
   X  L  M  W
   M  W  E  R
   I  B  E  Q

After column permutation: MEELWBWRQXMI

Matrix 2 (with key2 order [4, 2, 1, 3]):
   4  2  1  3
   M  E  E  L
   W  B  W  R
   Q  X  M  I

Final cipher text: EWMEBXLRIMWQ

DECRYPTION

Cipher text to decrypt: EWMEBXLRIMWQ
Cipher text in matrix form (arranged by columns):
   4  2  1  3
   M  E  E  L
   W  B  W  R
   Q  X  M  I

After reading row by row: MEELWBWRQXMI

Matrix after column arrangement:
   4  2  1  3
   X  L  M  W
   M  W  E  R
   I  B  E  Q

Before Caesar decryption: XLMWMWERIBEQ

Decrypted plain text: THISISANEXAM

VERIFICATION
Original:  THISISANEXAM
Decrypted: THISISANEXAM
Match: True


In [1]:
import numpy as np

class GraphCryptography:
    """
    Implementation of the cryptographic algorithm using Graph Theory
    as described in the research paper.
    """
    
    def __init__(self, adjacency_matrix, key1):
        """
        Initialize the cryptography system.
        
        Args:
            adjacency_matrix: 2D list/array representing the graph's adjacency matrix
            key1: Integer for Caesar cipher shift (e.g., 3 for +3 shift)
        """
        self.adj_matrix = np.array(adjacency_matrix)
        self.key1 = key1
        self.key2 = self._generate_key2_from_graph()
        
    def _generate_key2_from_graph(self):
        """
        Generate key2 from the adjacency matrix by counting connections.
        This follows the paper's example where the graph produces key2 = [4, 2, 1, 3]
        """
        # Count the number of 1s in each column (representing vertex connections)
        key2 = []
        for col_idx in range(self.adj_matrix.shape[1]):
            count = np.sum(self.adj_matrix[:, col_idx])
            key2.append(count)
        return key2
    
    def _caesar_encrypt(self, text):
        """Apply Caesar cipher encryption with key1."""
        result = ""
        for char in text:
            if char.isalpha():
                # Shift character by key1
                base = ord('A') if char.isupper() else ord('a')
                shifted = (ord(char) - base + self.key1) % 26
                result += chr(base + shifted)
            else:
                result += char
        return result
    
    def _caesar_decrypt(self, text):
        """Apply Caesar cipher decryption with key1."""
        result = ""
        for char in text:
            if char.isalpha():
                # Shift character back by key1
                base = ord('A') if char.isupper() else ord('a')
                shifted = (ord(char) - base - self.key1) % 26
                result += chr(base + shifted)
            else:
                result += char
        return result
    
    def _create_matrix(self, text, cols):
        """Create matrix from text with specified number of columns."""
        # Remove spaces for matrix creation
        text = text.replace(" ", "")
        rows = (len(text) + cols - 1) // cols  # Ceiling division
        
        # Pad text if necessary
        padded_text = text.ljust(rows * cols, 'X')
        
        # Create matrix
        matrix = []
        for i in range(rows):
            row = list(padded_text[i * cols:(i + 1) * cols])
            matrix.append(row)
        return matrix
    
    def _read_by_column_order(self, matrix, key_order):
        """Read matrix column by column according to key order."""
        result = ""
        # Sort indices by key values
        sorted_indices = sorted(range(len(key_order)), key=lambda k: key_order[k])
        
        for col_idx in sorted_indices:
            for row in matrix:
                if col_idx < len(row):
                    result += row[col_idx]
        return result
    
    def _read_by_row(self, matrix):
        """Read matrix row by row."""
        result = ""
        for row in matrix:
            result += ''.join(row)
        return result
    
    def encrypt(self, plaintext):
        """
        Encrypt plaintext using the algorithm from the paper.
        
        Args:
            plaintext: String to encrypt
            
        Returns:
            Encrypted cipher text
        """
        # Step 1: Remove spaces and convert to uppercase
        plaintext = plaintext.upper().replace(" ", "")
        
        # Step 2: Apply Caesar cipher with key1
        caesar_encrypted = self._caesar_encrypt(plaintext)
        print(f"After Caesar cipher (+{self.key1}): {caesar_encrypted}")
        
        # Step 3: Write in matrix form using key2 dimensions
        cols = len(self.key2)
        matrix1 = self._create_matrix(caesar_encrypted, cols)
        print(f"\nMatrix 1 (with key2 order {self.key2}):")
        self._print_matrix(matrix1, self.key2)
        
        # Step 4: Read row by row and permute columns
        permuted = self._read_by_column_order(matrix1, self.key2)
        print(f"\nAfter column permutation: {permuted}")
        
        # Step 5: Write permuted text in matrix form again
        matrix2 = self._create_matrix(permuted, cols)
        print(f"\nMatrix 2 (with key2 order {self.key2}):")
        self._print_matrix(matrix2, self.key2)
        
        # Step 6: Read column by column according to key2 order to get final cipher text
        ciphertext = self._read_by_column_order(matrix2, self.key2)
        print(f"\nFinal cipher text: {ciphertext}")
        
        return ciphertext
    
    def decrypt(self, ciphertext):
        """
        Decrypt cipher text using the algorithm from the paper.
        
        Args:
            ciphertext: String to decrypt
            
        Returns:
            Decrypted plain text
        """
        # Step 1: Write cipher text in matrix form
        cols = len(self.key2)
        
        # First, arrange the ciphertext by columns according to key2 order
        matrix1 = self._arrange_by_columns(ciphertext, self.key2)
        print(f"Cipher text in matrix form (arranged by columns):")
        self._print_matrix(matrix1, self.key2)
        
        # Step 2: Read row by row
        intermediate = self._read_by_row(matrix1)
        print(f"\nAfter reading row by row: {intermediate}")
        
        # Step 3: Arrange in matrix form column by column using key2
        matrix2 = self._arrange_by_columns(intermediate, self.key2)
        print(f"\nMatrix after column arrangement:")
        self._print_matrix(matrix2, self.key2)
        
        # Step 4: Read message row by row
        caesar_encrypted = self._read_by_row(matrix2)
        print(f"\nBefore Caesar decryption: {caesar_encrypted}")
        
        # Step 5: Apply Caesar decryption with key1
        plaintext = self._caesar_decrypt(caesar_encrypted)
        print(f"\nDecrypted plain text: {plaintext}")
        
        return plaintext
    
    def _arrange_by_columns(self, text, key_order):
        """Arrange text in matrix by filling columns according to key order."""
        cols = len(key_order)
        rows = len(text) // cols
        
        # Create empty matrix
        matrix = [['' for _ in range(cols)] for _ in range(rows)]
        
        # Sort indices by key values
        sorted_indices = sorted(range(len(key_order)), key=lambda k: key_order[k])
        
        # Fill columns according to sorted order
        text_idx = 0
        for col_idx in sorted_indices:
            for row_idx in range(rows):
                if text_idx < len(text):
                    matrix[row_idx][col_idx] = text[text_idx]
                    text_idx += 1
        
        return matrix
    
    def _print_matrix(self, matrix, key_header=None):
        """Print matrix in a formatted way."""
        if key_header:
            print("   " + "  ".join(map(str, key_header)))
        for row in matrix:
            print("   " + "  ".join(row))


# Example usage based on the paper
if __name__ == "__main__":
    # Define the adjacency matrix from the paper (Figure 1)
    adjacency_matrix = [
        [1, 1, 1, 0],  # V1
        [1, 0, 1, 0],  # V2
        [1, 1, 1, 1],  # V3
        [0, 0, 1, 1]   # V4
    ]
    
    # Key1 for Caesar cipher
    key1 = 4
    
    # Create cryptography instance
    crypto = GraphCryptography(adjacency_matrix, key1)
    
    print("="*60)
    print("GRAPH THEORY CRYPTOGRAPHY")
    print("="*60)
    print(f"\nAdjacency Matrix:")
    print(np.array(adjacency_matrix))
    print(f"\nGenerated Key2 from graph: {crypto.key2}")
    print(f"Key1 (Caesar shift): +{key1}")
    
    # Test with the example from the paper
    plaintext = "THIS IS AN EXAM"
    print(f"\n{'='*60}")
    print("ENCRYPTION")
    print("="*60)
    print(f"\nOriginal plaintext: {plaintext}")
    
    ciphertext = crypto.encrypt(plaintext)
    
    print(f"\n{'='*60}")
    print("DECRYPTION")
    print("="*60)
    print(f"\nCipher text to decrypt: {ciphertext}")
    
    decrypted = crypto.decrypt(ciphertext)
    
    print(f"\n{'='*60}")
    print("VERIFICATION")
    print("="*60)
    print(f"Original:  {plaintext.replace(' ', '')}")
    print(f"Decrypted: {decrypted}")
    print(f"Match: {plaintext.replace(' ', '').upper() == decrypted}")

GRAPH THEORY CRYPTOGRAPHY

Adjacency Matrix:
[[1 1 1 0]
 [1 0 1 0]
 [1 1 1 1]
 [0 0 1 1]]

Generated Key2 from graph: [3, 2, 4, 2]
Key1 (Caesar shift): +4

ENCRYPTION

Original plaintext: THIS IS AN EXAM
After Caesar cipher (+4): XLMWMWERIBEQ

Matrix 1 (with key2 order [3, 2, 4, 2]):
   3  2  4  2
   X  L  M  W
   M  W  E  R
   I  B  E  Q

After column permutation: LWBWRQXMIMEE

Matrix 2 (with key2 order [3, 2, 4, 2]):
   3  2  4  2
   L  W  B  W
   R  Q  X  M
   I  M  E  E

Final cipher text: WQMWMELRIBXE

DECRYPTION

Cipher text to decrypt: WQMWMELRIBXE
Cipher text in matrix form (arranged by columns):
   3  2  4  2
   L  W  B  W
   R  Q  X  M
   I  M  E  E

After reading row by row: LWBWRQXMIMEE

Matrix after column arrangement:
   3  2  4  2
   X  L  M  W
   M  W  E  R
   I  B  E  Q

Before Caesar decryption: XLMWMWERIBEQ

Decrypted plain text: THISISANEXAM

VERIFICATION
Original:  THISISANEXAM
Decrypted: THISISANEXAM
Match: True


In [5]:
import math
import numpy as np

class GraphCryptography:
    def __init__(self, adjacency_matrix, key1, padding_char='X'):
        self.adj_matrix = np.array(adjacency_matrix)
        self.key1 = key1
        self.key2 = self._generate_key2_from_graph()
        self.padding_char = padding_char

    def _generate_key2_from_graph(self):
        key2 = []
        for col_idx in range(self.adj_matrix.shape[1]):
            key2.append(int(np.sum(self.adj_matrix[:, col_idx])))
        return key2

    def _caesar_encrypt(self, text):
        result = ""
        for char in text:
            if char.isalpha():
                base = ord('A') if char.isupper() else ord('a')
                result += chr((ord(char) - base + self.key1) % 26 + base)
            else:
                result += char
        return result

    def _caesar_decrypt(self, text):
        result = ""
        for char in text:
            if char.isalpha():
                base = ord('A') if char.isupper() else ord('a')
                result += chr((ord(char) - base - self.key1) % 26 + base)
            else:
                result += char
        return result

    def _create_matrix(self, text, cols):
        text = text.replace(' ', '')
        rows = (len(text) + cols - 1) // cols
        padded_text = text.ljust(rows * cols, self.padding_char)
        matrix = []
        for i in range(rows):
            matrix.append(list(padded_text[i*cols:(i+1)*cols]))
        return matrix

    def _read_by_column_order(self, matrix, key_order):
        result = ""
        sorted_indices = sorted(range(len(key_order)), key=lambda k: key_order[k])
        for col_idx in sorted_indices:
            for row in matrix:
                if col_idx < len(row):
                    result += row[col_idx]
        return result

    def _read_by_row(self, matrix):
        return ''.join(''.join(row) for row in matrix)

    def _print_matrix(self, matrix, key_header=None):
        """Print matrix in a formatted way."""
        if key_header:
            print("   " + "  ".join(map(str, key_header)))
        for row in matrix:
            print("   " + "  ".join(row))

    def _arrange_by_columns(self, text, key_order):
        """
        Robust arrange: compute rows with ceiling and fill missing cells
        with padding so columns always have uniform height.
        This fixes issues when len(text) is not divisible by cols (e.g., primes).
        """
        cols = len(key_order)
        rows = (len(text) + cols - 1) // cols  # ceiling

        # Initialize matrix filled with padding_char
        matrix = [[self.padding_char for _ in range(cols)] for _ in range(rows)]

        # Determine the order of columns according to key_order
        sorted_indices = sorted(range(len(key_order)), key=lambda k: key_order[k])

        text_idx = 0
        for col_idx in sorted_indices:
            for row_idx in range(rows):
                if text_idx < len(text):
                    matrix[row_idx][col_idx] = text[text_idx]
                    text_idx += 1
                else:
                    # leave padding
                    matrix[row_idx][col_idx] = self.padding_char
        return matrix

    def encrypt(self, plaintext):
        plaintext = plaintext.replace(' ', '').upper()
        original_length = len(plaintext)
        
        print(f"Step 0 - Original plaintext: {plaintext}")
        print(f"Original length: {original_length}\n")

        caesar = self._caesar_encrypt(plaintext)
        print(f"Step 1 - After Caesar cipher (+{self.key1}): {caesar}\n")


        cols = len(self.key2)
        matrix1 = self._create_matrix(caesar, cols)
        print(f"Step 2 - Matrix 1 (Caesar text arranged with {cols} columns):")
        print(f"\nAdjacency Matrix: \n")
        print(np.array(adjacency_matrix))
        print(f"Key2 order: {self.key2}")
        self._print_matrix(matrix1, self.key2)
        
        step1 = self._read_by_column_order(matrix1, self.key2)
        print(f"\nStep 3 - After reading columns by key2 order: {step1}\n")

        matrix2 = self._create_matrix(step1, cols)
        print(f"Step 4 - Matrix 2 (permuted text arranged with {cols} columns):")
        self._print_matrix(matrix2, self.key2)
        
        ciphertext = self._read_by_column_order(matrix2, self.key2)
        print(f"\nStep 5 - Final ciphertext: {ciphertext}\n")

        return ciphertext, original_length

    def decrypt(self, ciphertext, original_length):
        cols = len(self.key2)
        
        print(f"\n{'='*60}")
        print("DECRYPTION PROCESS")
        print(f"{'='*60}")
        print(f"\nStep 0 - Ciphertext to decrypt: {ciphertext}\n")

        matrix1 = self._arrange_by_columns(ciphertext, self.key2)
        print(f"Step 1 - Matrix 1 (ciphertext arranged by columns):")
        print(f"Key2 order: {self.key2}")
        self._print_matrix(matrix1, self.key2)
        
        step1 = self._read_by_row(matrix1)
        print(f"\nStep 2 - After reading row by row: {step1}\n")

        matrix2 = self._arrange_by_columns(step1, self.key2)
        print(f"Step 3 - Matrix 2 (permuted text arranged by columns):")
        self._print_matrix(matrix2, self.key2)
        
        caesar_text = self._read_by_row(matrix2)
        print(f"\nStep 4 - Before Caesar decryption: {caesar_text}\n")

        plaintext = self._caesar_decrypt(caesar_text)
        plaintext = plaintext[:original_length]
        print(f"Step 5 - After Caesar decryption (-{self.key1}): {plaintext}\n")
        
        return plaintext


if __name__ == '__main__':
    
    adjacency_matrix = [
        [1,1,1,1],
        [1,0,0,1],
        [1,0,0,0],
        [1,1,0,1]
    ]
    key1 = 4
    crypto = GraphCryptography(adjacency_matrix, key1)


    plaintext = "THIS IS AN EXAMO"
    print(f"\n{'='*60}")
    print("ENCRYPTION")
    print("="*60)
   


    ciphertext, orig_len = crypto.encrypt(plaintext)
    print('Ciphertext:', ciphertext)

    decrypted = crypto.decrypt(ciphertext, orig_len)

    print(f"\n{'='*60}")
    print("VERIFICATION")
    print("="*60)
    print(f"Original:  {plaintext.replace(' ', '')}")
    print(f"Decrypted: {decrypted}")
    print(f"Match: {plaintext.replace(' ', '').upper() == decrypted}")

 



ENCRYPTION
Step 0 - Original plaintext: THISISANEXAMO
Original length: 13

Step 1 - After Caesar cipher (+4): XLMWMWERIBEQS

Step 2 - Matrix 1 (Caesar text arranged with 4 columns):

Adjacency Matrix: 

[[1 1 1 1]
 [1 0 0 1]
 [1 0 0 0]
 [1 1 0 1]]
Key2 order: [4, 2, 1, 3]
   4  2  1  3
   X  L  M  W
   M  W  E  R
   I  B  E  Q
   S  X  X  X

Step 3 - After reading columns by key2 order: MEEXLWBXWRQXXMIS

Step 4 - Matrix 2 (permuted text arranged with 4 columns):
   4  2  1  3
   M  E  E  X
   L  W  B  X
   W  R  Q  X
   X  M  I  S

Step 5 - Final ciphertext: EBQIEWRMXXXSMLWX

Ciphertext: EBQIEWRMXXXSMLWX

DECRYPTION PROCESS

Step 0 - Ciphertext to decrypt: EBQIEWRMXXXSMLWX

Step 1 - Matrix 1 (ciphertext arranged by columns):
Key2 order: [4, 2, 1, 3]
   4  2  1  3
   M  E  E  X
   L  W  B  X
   W  R  Q  X
   X  M  I  S

Step 2 - After reading row by row: MEEXLWBXWRQXXMIS

Step 3 - Matrix 2 (permuted text arranged by columns):
   4  2  1  3
   X  L  M  W
   M  W  E  R
   I  B  E  Q
   S

In [28]:
import math
import numpy as np

class GraphCryptography:
    def __init__(self, adjacency_matrix, key1, padding_char='X'):
        self.adj_matrix = np.array(adjacency_matrix)
        self.key1 = key1
        self.key2 = self._generate_key2_from_graph()
        self.padding_char = padding_char

    def _generate_key2_from_graph(self):
        key2 = []
        for col_idx in range(self.adj_matrix.shape[1]):
            key2.append(int(np.sum(self.adj_matrix[:, col_idx])))
        return key2

    def _caesar_encrypt(self, text):
        result = ""
        for char in text:
            if char.isalpha():
                base = ord('A') if char.isupper() else ord('a')
                result += chr((ord(char) - base + self.key1) % 26 + base)
            else:
                result += char
        return result

    def _caesar_decrypt(self, text):
        result = ""
        for char in text:
            if char.isalpha():
                base = ord('A') if char.isupper() else ord('a')
                result += chr((ord(char) - base - self.key1) % 26 + base)
            else:
                result += char
        return result

    def _create_matrix(self, text, cols):
        text = text.replace(' ', '')
        rows = (len(text) + cols - 1) // cols
        padded_text = text.ljust(rows * cols, self.padding_char)
        matrix = []
        for i in range(rows):
            matrix.append(list(padded_text[i*cols:(i+1)*cols]))
        return matrix

    def _read_by_column_order(self, matrix, key_order):
        result = ""
        sorted_indices = sorted(range(len(key_order)), key=lambda k: key_order[k])
        for col_idx in sorted_indices:
            for row in matrix:
                if col_idx < len(row):
                    result += row[col_idx]
        return result

    def _read_by_row(self, matrix):
        return ''.join(''.join(row) for row in matrix)

    def _print_matrix(self, matrix, key_header=None):
        """Print matrix in a formatted way."""
        if key_header:
            print("   " + "  ".join(map(str, key_header)))
        for row in matrix:
            print("   " + "  ".join(row))

    def _arrange_by_columns(self, text, key_order):
        """
        Robust arrange: compute rows with ceiling and fill missing cells
        with padding so columns always have uniform height.
        This fixes issues when len(text) is not divisible by cols (e.g., primes).
        """
        cols = len(key_order)
        rows = (len(text) + cols - 1) // cols  # ceiling

        # Initialize matrix filled with padding_char
        matrix = [[self.padding_char for _ in range(cols)] for _ in range(rows)]

        # Determine the order of columns according to key_order
        sorted_indices = sorted(range(len(key_order)), key=lambda k: key_order[k])

        text_idx = 0
        for col_idx in sorted_indices:
            for row_idx in range(rows):
                if text_idx < len(text):
                    matrix[row_idx][col_idx] = text[text_idx]
                    text_idx += 1
                else:
                    # leave padding
                    matrix[row_idx][col_idx] = self.padding_char
        return matrix

    def encrypt(self, plaintext):
        plaintext = plaintext.replace(' ', '').upper()
        original_length = len(plaintext)
        
        print(f"Step 0 - Original plaintext: {plaintext}")
        print(f"Original length: {original_length}\n")

        caesar = self._caesar_encrypt(plaintext)
        print(f"Step 1 - After Caesar cipher (+{self.key1}): {caesar}\n")

        cols = len(self.key2)
        matrix1 = self._create_matrix(caesar, cols)
        print(f"Step 2 - Matrix 1 (Caesar text arranged with {cols} columns):")
        print(f"Key2 order: {self.key2}")
        self._print_matrix(matrix1, self.key2)
        
        step1 = self._read_by_column_order(matrix1, self.key2)
        print(f"\nStep 3 - After reading columns by key2 order: {step1}\n")

        matrix2 = self._create_matrix(step1, cols)
        print(f"Step 4 - Matrix 2 (permuted text arranged with {cols} columns):")
        self._print_matrix(matrix2, self.key2)
        
        ciphertext = self._read_by_column_order(matrix2, self.key2)
        print(f"\nStep 5 - Final ciphertext: {ciphertext}\n")

        return ciphertext, original_length

    def decrypt(self, ciphertext, original_length):
        cols = len(self.key2)
        
        print(f"\n{'='*60}")
        print("DECRYPTION PROCESS")
        print(f"{'='*60}")
        print(f"\nStep 0 - Ciphertext to decrypt: {ciphertext}\n")

        matrix1 = self._arrange_by_columns(ciphertext, self.key2)
        print(f"Step 1 - Matrix 1 (ciphertext arranged by columns):")
        print(f"Key2 order: {self.key2}")
        self._print_matrix(matrix1, self.key2)
        
        step1 = self._read_by_row(matrix1)
        print(f"\nStep 2 - After reading row by row: {step1}\n")

        matrix2 = self._arrange_by_columns(step1, self.key2)
        print(f"Step 3 - Matrix 2 (permuted text arranged by columns):")
        self._print_matrix(matrix2, self.key2)
        
        caesar_text = self._read_by_row(matrix2)
        print(f"\nStep 4 - Before Caesar decryption: {caesar_text}\n")

        plaintext = self._caesar_decrypt(caesar_text)
        plaintext = plaintext[:original_length]
        print(f"Step 5 - After Caesar decryption (-{self.key1}): {plaintext}\n")
        
        return plaintext


if __name__ == '__main__':
    
    adjacency_matrix = [
        [1, 1, 1, 1, 0, 1, 0],  # V1
        [1, 1, 1, 1, 1, 0, 1],  # V2
        [1, 1, 0, 1, 0, 0, 1],  # V3
        [0, 1, 0, 0, 0, 1, 1],
        [0, 1, 1, 0, 0, 0, 1],  # V2
        [1, 1, 0, 1, 0, 0, 1],  # V3
        [0, 1, 0, 1, 0, 0, 1]   # V4
    ]
    
    key1 = 4
    crypto = GraphCryptography(adjacency_matrix, key1)


    plaintext = "THIS IS AN EXAMO"
    print(f"\n{'='*60}")
    print("ENCRYPTION")
    print("="*60)
   


    ciphertext, orig_len = crypto.encrypt(plaintext)
    print('Ciphertext:', ciphertext)

    decrypted = crypto.decrypt(ciphertext, orig_len)

    print(f"\n{'='*60}")
    print("VERIFICATION")
    print("="*60)
    print(f"Original:  {plaintext.replace(' ', '')}")
    print(f"Decrypted: {decrypted}")
    print(f"Match: {plaintext.replace(' ', '').upper() == decrypted}")

 



ENCRYPTION
Step 0 - Original plaintext: THISISANEXAMO
Original length: 13

Step 1 - After Caesar cipher (+4): XLMWMWERIBEQS

Step 2 - Matrix 1 (Caesar text arranged with 7 columns):
Key2 order: [4, 7, 3, 5, 1, 2, 6]
   4  7  3  5  1  2  6
   X  L  M  W  M  W  E
   R  I  B  E  Q  S  X

Step 3 - After reading columns by key2 order: MQWSMBXRWEEXLI

Step 4 - Matrix 2 (permuted text arranged with 7 columns):
   4  7  3  5  1  2  6
   M  Q  W  S  M  B  X
   R  W  E  E  X  L  I

Step 5 - Final ciphertext: MXBLWEMRSEXIQW

Ciphertext: MXBLWEMRSEXIQW

DECRYPTION PROCESS

Step 0 - Ciphertext to decrypt: MXBLWEMRSEXIQW

Step 1 - Matrix 1 (ciphertext arranged by columns):
Key2 order: [4, 7, 3, 5, 1, 2, 6]
   4  7  3  5  1  2  6
   M  Q  W  S  M  B  X
   R  W  E  E  X  L  I

Step 2 - After reading row by row: MQWSMBXRWEEXLI

Step 3 - Matrix 2 (permuted text arranged by columns):
   4  7  3  5  1  2  6
   X  L  M  W  M  W  E
   R  I  B  E  Q  S  X

Step 4 - Before Caesar decryption: XLMWMWERIBEQSX



In [29]:
import math
import numpy as np

class GraphCryptography:
    def __init__(self, adjacency_matrix, key1, padding_char='X'):
        self.adj_matrix = np.array(adjacency_matrix)
        self.key1 = key1
        self.key2 = self._generate_key2_from_graph()
        self.padding_char = padding_char

    def _generate_key2_from_graph(self):
        key2 = []
        for col_idx in range(self.adj_matrix.shape[1]):
            key2.append(int(np.sum(self.adj_matrix[:, col_idx])))
        return key2

    def _caesar_encrypt(self, text):
        result = ""
        for char in text:
            if char.isalpha():
                base = ord('A') if char.isupper() else ord('a')
                result += chr((ord(char) - base + self.key1) % 26 + base)
            else:
                result += char
        return result

    def _caesar_decrypt(self, text):
        result = ""
        for char in text:
            if char.isalpha():
                base = ord('A') if char.isupper() else ord('a')
                result += chr((ord(char) - base - self.key1) % 26 + base)
            else:
                result += char
        return result

    def _create_matrix(self, text, cols):
        text = text.replace(' ', '')
        rows = (len(text) + cols - 1) // cols
        padded_text = text.ljust(rows * cols, self.padding_char)
        matrix = []
        for i in range(rows):
            matrix.append(list(padded_text[i*cols:(i+1)*cols]))
        return matrix

    def _read_by_column_order(self, matrix, key_order):
        result = ""
        sorted_indices = sorted(range(len(key_order)), key=lambda k: key_order[k])
        for col_idx in sorted_indices:
            for row in matrix:
                if col_idx < len(row):
                    result += row[col_idx]
        return result

    def _read_by_row(self, matrix):
        return ''.join(''.join(row) for row in matrix)

    def _print_matrix(self, matrix, key_header=None):
        """Print matrix in a formatted way."""
        if key_header:
            print("   " + "  ".join(map(str, key_header)))
        for row in matrix:
            print("   " + "  ".join(row))

    def _arrange_by_columns(self, text, key_order):
        """
        Robust arrange: compute rows with ceiling and fill missing cells
        with padding so columns always have uniform height.
        This fixes issues when len(text) is not divisible by cols (e.g., primes).
        """
        cols = len(key_order)
        rows = (len(text) + cols - 1) // cols  # ceiling

        # Initialize matrix filled with padding_char
        matrix = [[self.padding_char for _ in range(cols)] for _ in range(rows)]

        # Determine the order of columns according to key_order
        sorted_indices = sorted(range(len(key_order)), key=lambda k: key_order[k])

        text_idx = 0
        for col_idx in sorted_indices:
            for row_idx in range(rows):
                if text_idx < len(text):
                    matrix[row_idx][col_idx] = text[text_idx]
                    text_idx += 1
                else:
                    # leave padding
                    matrix[row_idx][col_idx] = self.padding_char
        return matrix

    def encrypt(self, plaintext):
        plaintext = plaintext.replace(' ', '').upper()
        original_length = len(plaintext)
        
        print(f"Step 0 - Original plaintext: {plaintext}")
        print(f"Original length: {original_length}\n")

        caesar = self._caesar_encrypt(plaintext)
        print(f"Step 1 - After Caesar cipher (+{self.key1}): {caesar}\n")

        cols = len(self.key2)
        matrix1 = self._create_matrix(caesar, cols)
        print(f"Step 2 - Matrix 1 (Caesar text arranged with {cols} columns):")
        print(f"Key2 order: {self.key2}")
        self._print_matrix(matrix1, self.key2)
        
        step1 = self._read_by_column_order(matrix1, self.key2)
        print(f"\nStep 3 - After reading columns by key2 order: {step1}\n")

        matrix2 = self._create_matrix(step1, cols)
        print(f"Step 4 - Matrix 2 (permuted text arranged with {cols} columns):")
        self._print_matrix(matrix2, self.key2)
        
        ciphertext = self._read_by_column_order(matrix2, self.key2)
        print(f"\nStep 5 - Final ciphertext: {ciphertext}\n")

        return ciphertext, original_length

    def decrypt(self, ciphertext, original_length):
        cols = len(self.key2)
        
        print(f"\n{'='*60}")
        print("DECRYPTION PROCESS")
        print(f"{'='*60}")
        print(f"\nStep 0 - Ciphertext to decrypt: {ciphertext}\n")

        matrix1 = self._arrange_by_columns(ciphertext, self.key2)
        print(f"Step 1 - Matrix 1 (ciphertext arranged by columns):")
        print(f"Key2 order: {self.key2}")
        self._print_matrix(matrix1, self.key2)
        
        step1 = self._read_by_row(matrix1)
        print(f"\nStep 2 - After reading row by row: {step1}\n")

        matrix2 = self._arrange_by_columns(step1, self.key2)
        print(f"Step 3 - Matrix 2 (permuted text arranged by columns):")
        self._print_matrix(matrix2, self.key2)
        
        caesar_text = self._read_by_row(matrix2)
        print(f"\nStep 4 - Before Caesar decryption: {caesar_text}\n")

        plaintext = self._caesar_decrypt(caesar_text)
        plaintext = plaintext[:original_length]
        print(f"Step 5 - After Caesar decryption (-{self.key1}): {plaintext}\n")
        
        return plaintext


if __name__ == '__main__':
    
    adjacency_matrix = [
        [1, 1, 1, 1, 0, 1, 0],  # V1
        [1, 1, 1, 1, 1, 0, 1],  # V2
        [1, 1, 0, 1, 0, 0, 1],  # V3
        [0, 1, 0, 0, 0, 1, 1],
        [0, 1, 1, 0, 0, 0, 1],  # V2
        [1, 1, 0, 1, 0, 0, 1],  # V3
        [0, 1, 0, 1, 0, 0, 1]   # V4
    ]
    
    key1 = 4
    crypto = GraphCryptography(adjacency_matrix, key1)


    plaintext = "THIS IS AN EXAM"
    print(f"\n{'='*60}")
    print("ENCRYPTION")
    print("="*60)
   


    ciphertext, orig_len = crypto.encrypt(plaintext)
    print('Ciphertext:', ciphertext)

    decrypted = crypto.decrypt(ciphertext, orig_len)

    print(f"\n{'='*60}")
    print("VERIFICATION")
    print("="*60)
    print(f"Original:  {plaintext.replace(' ', '')}")
    print(f"Decrypted: {decrypted}")
    print(f"Match: {plaintext.replace(' ', '').upper() == decrypted}")

 



ENCRYPTION
Step 0 - Original plaintext: THISISANEXAM
Original length: 12

Step 1 - After Caesar cipher (+4): XLMWMWERIBEQ

Step 2 - Matrix 1 (Caesar text arranged with 7 columns):
Key2 order: [4, 7, 3, 5, 1, 2, 6]
   4  7  3  5  1  2  6
   X  L  M  W  M  W  E
   R  I  B  E  Q  X  X

Step 3 - After reading columns by key2 order: MQWXMBXRWEEXLI

Step 4 - Matrix 2 (permuted text arranged with 7 columns):
   4  7  3  5  1  2  6
   M  Q  W  X  M  B  X
   R  W  E  E  X  L  I

Step 5 - Final ciphertext: MXBLWEMRXEXIQW

Ciphertext: MXBLWEMRXEXIQW

DECRYPTION PROCESS

Step 0 - Ciphertext to decrypt: MXBLWEMRXEXIQW

Step 1 - Matrix 1 (ciphertext arranged by columns):
Key2 order: [4, 7, 3, 5, 1, 2, 6]
   4  7  3  5  1  2  6
   M  Q  W  X  M  B  X
   R  W  E  E  X  L  I

Step 2 - After reading row by row: MQWXMBXRWEEXLI

Step 3 - Matrix 2 (permuted text arranged by columns):
   4  7  3  5  1  2  6
   X  L  M  W  M  W  E
   R  I  B  E  Q  X  X

Step 4 - Before Caesar decryption: XLMWMWERIBEQXX

St